In [135]:
import requests

import json
from pathlib import Path
from bs4 import BeautifulSoup, Comment
from time import sleep
import selenium
import re
from pathlib import Path

In [125]:
def get_content(url):
    sleep(1)
    resp = requests.get(url)

    resp.encoding = resp.apparent_encoding
    try:
        soup = BeautifulSoup(resp.text, "lxml")
    except:
        soup = BeautifulSoup(resp.text, "html5lib")

    meta = soup.find_all('meta',
                     attrs={"name":"robots"},
                     content=lambda x: "nofollow" in str(x).lower() or "noarchive" in str(x).lower())

    s = str(resp.headers.get("X-Robots-Tag"))
    if meta != [] and ("nofollow" in s) and ("noarchive" in s):
        raise Exception

    for comment in soup(text=lambda x: isinstance(x, Comment)):
        comment.extract()

    if soup.blockquote is not None:
        soup.blockquote.decompose()

    return soup

In [78]:
base_url = "http://radical-d.extrem.ne.jp/toho/serihu/"

In [104]:
def get_stories():
    soup = get_content(base_url + "serihu.htm")
    return [i.get("href") for i in soup.find("td", class_="line").find_all("a")[1:]]

In [105]:
get_stories()

['kouma00.htm',
 'youyou00.htm',
 'suimu00.htm',
 'eiya00.htm',
 'kaei00.htm',
 'fujin00.htm',
 'hisou00.htm',
 'tirei00.htm',
 'seiren00.htm',
 'ds00.htm',
 'yousei00.htm',
 'sinrei00.htm',
 'sinki00.htm',
 'kishin00.htm',
 'da00.htm',
 'sinpi00.htm',
 'kanju00.htm',
 'hyoui00.htm',
 'tenkuu00.htm',
 'hnd00.htm',
 'kikei00.htm']

In [106]:
def get_charas(url):
    soup = get_content(base_url+url)
    return [i.get("href") for i in soup.td.find_all("a")]

In [107]:
get_charas("kikei00.htm")

['kikei01.htm',
 'kikei02.htm',
 'kikei03.htm',
 'kikei04.htm',
 'kikei05.htm',
 'kikei06.htm',
 'kikei07.htm',
 'kikei08.htm',
 'kikei09.htm']

In [122]:
def get_serif(url):
    soup = get_content(base_url+url)
    serif = []
    for i in soup.find_all("pre"):
        serif.extend([re.sub("\u3000|\n", " ",  j.text).replace("：", ", ") for j in i.find_all("font")])
    return serif

In [123]:
get_serif("kikei01.htm")

['霊夢, 動物霊だらけね。     でもこんなところで手こずっている場合じゃないわ。',
 '瓔花, そこのお前！     こんなところで暴れるから積み石がみんな崩れちゃったよ。',
 '霊夢, 積み石？ そんなの邪魔だから崩したのよ。',
 '瓔花, 酷い。子供達が頑張って積んだのに……。     お前なんて石に当たって死んじまえ！',
 '瓔花, ぐぐぐ……強い。',
 '霊夢, 目的地はあの地獄よ。こんなあの世の入り口で手こずる訳にはいかないわ。',
 '瓔花, おや？ 地獄に行くのね。     あんたなんか地獄に行くのがお似合いだもんね。',
 '霊夢, どうとでも言いな。',
 '霊夢, いつになったら向こう岸に着くのかな。',
 '潤美, 三途の河（こんなところ）に生身の人間とは、凶暴な魚に引きずり込まれたいのか？',
 '霊夢, その角は……鬼？',
 '潤美, まあそんなようなもんだ。     今なら安全に送り返してやっても良いんだが……。',
 '霊夢, 結構よ。魚なんか、その場で捌いてやるから。',
 '潤美, そうか、なら丁度良い。遠慮無く愛しの古代魚達の餌にしてやろう。',
 '潤美, 道に迷ったわけではないのね。',
 '霊夢, 私の目的地は地獄なの。その為に河を渡っているだけよ。',
 '潤美, 生きたまま地獄に行くとは。想像するだけで身震いがするよ。',
 '霊夢, おや？ あんた、てっきり地獄の鬼かと思ったわ。',
 '久侘歌, 閻魔様から話は聞いています。容赦無く、貴方を試しなさいってね！',
 '久侘歌, 地獄へ行こうとしているだけありますね。中々の腕前でございます。',
 '霊夢, あんたは一体……？',
 '久侘歌, 私は、久侘歌（くたか）。地獄の関所を見張っています。',
 '霊夢, なるほど、地獄の門番ね。じゃあ話は簡単だわ。     私はその地獄に行きたいのよ。通してくれる？',
 '久侘歌, ふっふっふ。閻魔様は地獄に通しても良いと言っていましたが、      私はそんな残酷なことは好まないので……ここで大人しく帰って頂きたいと思います！',
 '久侘歌, お、お見事です。そこまでの覚悟があるのなら、もう止めません。      地獄の業火で焼かれようと、針のむしろに叩き付けられようとも、私は貴方のことを忘れます。鳥頭

In [128]:
s_all = []
for s in get_stories():
    print(s)
    for c in get_charas(s):
        s_all.extend(get_serif(c))

kouma00.htm
youyou00.htm
suimu00.htm
eiya00.htm
kaei00.htm
fujin00.htm
hisou00.htm
tirei00.htm
seiren00.htm
ds00.htm
yousei00.htm
sinrei00.htm
sinki00.htm
kishin00.htm
da00.htm
sinpi00.htm
kanju00.htm
hyoui00.htm
tenkuu00.htm
hnd00.htm
kikei00.htm


In [130]:
len(s_all)

7923

In [142]:
reimu = [s.split(",")[1] for s in s_all if (s.split(",")[0] == "霊夢")]

In [143]:
Path("reimu.csv").write_text("\n".join(reimu))

31755

In [144]:
import pandas as pd

In [145]:
pd.read_csv("reimu.csv", header=None).head()

,0
0,気持ちいいわね。毎回、昼間に出発して悪霊が少ないから、夜に出てみたんだけど……どこに行って...
1,って、あんた誰？
2,人は暗いところでは物が良く見えないのよ。
3,それは取って食べたりしてもいいのよ。
4,で、邪魔なんですけど。


In [147]:
Path("all.csv").write_text("\n".join(s_all))

294216

In [148]:
pd.read_csv("all.csv", header=None).head()

,0,1
0,霊夢,気持ちいいわね。毎回、昼間に出発して悪霊が少ないから、夜に出てみたんだけど……どこに行って...
1,ルーミア,そうなのよね〜。お化けも出るし、たまんないわ。
2,霊夢,って、あんた誰？
3,ルーミア,さっき会ったじゃない。あんた、もしかして鳥目？
4,霊夢,人は暗いところでは物が良く見えないのよ。


In [150]:
marisa = [s.split(",")[1] for s in s_all if (s.split(",")[0] == "魔理沙")]

In [152]:
Path("marisa.csv").write_text("\n".join(marisa))

30901

In [153]:
len(marisa)

1103

In [154]:
len(reimu)

1128